In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [67]:
ens = pd.read_excel("ENs_2006-2019.xlsx", index_col=0)

In [68]:
ens.head(3)

,EventDescription,SiteName,RegionNumber,StateCode,CityName,CountyName,UnitInd1,UnitInd2,UnitInd3,TimeZone,...,InitPower1,InitPower2,InitPower3,Imode1,Imode2,Imode3,Fmode1,Fmode2,Fmode3,EventText
EventNotificationNumber,,,,,,,,,,,,,,,,,,,,,
41128,Power Reactor,PALISADES,3.0,MI,COVERT,VANBUREN,1.0,NaN,NaN,EDT,...,0.0,0.0,0.0,Refueling,NaN,NaN,Refueling,NaN,NaN,REACTOR VESSEL HEAD NOZZLE CRACKING
41377,Power Reactor,PRAIRIE ISLAND,3.0,MN,WELCH,GOODHUE,1.0,NaN,NaN,CST,...,88.0,0.0,0.0,Power Operation,NaN,NaN,Power Operation,NaN,NaN,4160 VOLT RELAYING AND METERING SINGLE FAILURE...
41755,Power Reactor,SUMMER,2.0,SC,JENKINSVILLE,FAIRFIELD,1.0,NaN,NaN,EDT,...,100.0,0.0,0.0,Power Operation,NaN,NaN,Power Operation,NaN,NaN,TWENTY-EIGHT EARLY WARNING SIRENS TEMPORARILY ...


In [69]:
#eliminate columns that aren't useful for this analysisedn
ens_clean = ens[["SiteName","RegionNumber","EventDate","CfrDesc1",
                "CfrDesc2","CfrDesc3","CfrDesc4","ScramType1", "ScramType2", "ScramType3","EventText"]]
ens_clean.head(3)

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText
EventNotificationNumber,,,,,,,,,,,
41128,PALISADES,3.0,2004-10-16,50.72(b)(3)(ii)(A) - DEGRADED CONDITION,NaN,NaN,NaN,N,N,N,REACTOR VESSEL HEAD NOZZLE CRACKING
41377,PRAIRIE ISLAND,3.0,2005-02-05,50.72(b)(3)(ii)(B) - UNANALYZED CONDITION,NaN,NaN,NaN,N,N,N,4160 VOLT RELAYING AND METERING SINGLE FAILURE...
41755,SUMMER,2.0,2005-06-07,50.72(b)(3)(xiii) - LOSS COMM/ASMT/RESPONSE,NaN,NaN,NaN,N,N,N,TWENTY-EIGHT EARLY WARNING SIRENS TEMPORARILY ...


In [72]:
ens_clean.shape

(6399, 12)

In [71]:
#datasets all need a year column and a site column
#another layer would also use a quarter column, but not implemented here yet
ens_clean["Year"] = ens_clean["EventDate"].map(lambda x: x.year)

C:\Users\becbo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
ens_clean.head(3)

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year
EventNotificationNumber,,,,,,,,,,,,
41128,PALISADES,3.0,2004-10-16,50.72(b)(3)(ii)(A) - DEGRADED CONDITION,NaN,NaN,NaN,N,N,N,REACTOR VESSEL HEAD NOZZLE CRACKING,2004.0
41377,PRAIRIE ISLAND,3.0,2005-02-05,50.72(b)(3)(ii)(B) - UNANALYZED CONDITION,NaN,NaN,NaN,N,N,N,4160 VOLT RELAYING AND METERING SINGLE FAILURE...,2005.0
41755,SUMMER,2.0,2005-06-07,50.72(b)(3)(xiii) - LOSS COMM/ASMT/RESPONSE,NaN,NaN,NaN,N,N,N,TWENTY-EIGHT EARLY WARNING SIRENS TEMPORARILY ...,2005.0


In [74]:
pd.DataFrame(ens_clean.dtypes, columns=['DataType'])

,DataType
SiteName,object
RegionNumber,float64
EventDate,datetime64[ns]
CfrDesc1,object
CfrDesc2,object
CfrDesc3,object
CfrDesc4,float64
ScramType1,object
ScramType2,object
ScramType3,object


In [75]:
#Need to use uniform site names across all the datasets that will be merged
ens_clean["SiteName"] = ens_clean["SiteName"].map({
        'FT CALHOUN': 'FORT CALHOUN',
        'ARKANSAS NUCLEAR': 'ARKANSAS NUCLEAR ONE',
        'SOUTH TEXAS': 'SOUTH TEXAS PROJECT',
        'COLUMBIA GENERATING STATION': 'COLUMBIA',
        }).fillna(ens_clean["SiteName"])

C:\Users\becbo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [76]:
ens_clean[ens_clean["SiteName"] == "FT CALHOUN"]

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year
EventNotificationNumber,,,,,,,,,,,,


In [77]:
ens_clean.head(3)

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year
EventNotificationNumber,,,,,,,,,,,,
41128,PALISADES,3.0,2004-10-16,50.72(b)(3)(ii)(A) - DEGRADED CONDITION,NaN,NaN,NaN,N,N,N,REACTOR VESSEL HEAD NOZZLE CRACKING,2004.0
41377,PRAIRIE ISLAND,3.0,2005-02-05,50.72(b)(3)(ii)(B) - UNANALYZED CONDITION,NaN,NaN,NaN,N,N,N,4160 VOLT RELAYING AND METERING SINGLE FAILURE...,2005.0
41755,SUMMER,2.0,2005-06-07,50.72(b)(3)(xiii) - LOSS COMM/ASMT/RESPONSE,NaN,NaN,NaN,N,N,N,TWENTY-EIGHT EARLY WARNING SIRENS TEMPORARILY ...,2005.0


In [78]:
ens_clean["Year"].unique()

array([2004., 2005., 2006., 2003., 2007., 1976., 2008., 2009., 2010.,
       2011., 2012.,   nan, 2013., 2103., 2014., 2015., 2016., 2017.,
       2018., 2019.])

In [79]:
ens_clean["Year"].fillna(value = 2013, inplace = True)

C:\Users\becbo\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [80]:
ens_clean[ens_clean["Year"].isnull()]

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year
EventNotificationNumber,,,,,,,,,,,,


In [81]:
ens_clean[ens_clean["Year"] == 2103]

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year
EventNotificationNumber,,,,,,,,,,,,
49565,SUSQUEHANNA,1.0,2103-11-20,50.72(b)(3)(v)(C) - POT UNCNTRL RAD REL,NaN,NaN,NaN,N,N,N,SECONDARY CONTAINMENT FAILS DRAWDOWN SURVEILLA...,2103.0


In [82]:
ens_clean.at[49565,"Year"] = 2013

In [83]:
ens_clean = ens_clean[(ens_clean.Year != 1976) & (ens_clean.Year != 2003) & (ens_clean.Year != 2004) & (ens_clean.Year != 2005)]

In [84]:
ens_clean["Year"].unique()

array([2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014.,
       2015., 2016., 2017., 2018., 2019.])

In [85]:
ens_clean["Year"] = ens_clean["Year"].apply(np.int64)

In [86]:
ens_clean["Year"].head()

EventNotificationNumber
42241    2006
42242    2006
42243    2006
42245    2006
42246    2006
Name: Year, dtype: int64

In [87]:
ens_clean["RegionNumber"].unique()

array([ 3.,  4.,  1.,  2.,  0., nan])

In [88]:
ens_clean[ens_clean["RegionNumber"].isnull()]

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year
EventNotificationNumber,,,,,,,,,,,,
ions are repaired. The expanded scope inspection activity is continuing,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013


In [89]:
ens_clean = ens_clean[(ens_clean.RegionNumber != 0)]

In [92]:
ens_clean.dropna(subset=['RegionNumber'], inplace = True)

In [93]:
ens_clean.shape

(6383, 12)

In [94]:
ens_clean["RegionNumber"] = ens_clean["RegionNumber"].apply(np.int64)

In [95]:
ens_clean["RegionNumber"].head(3)

EventNotificationNumber
42241    3
42242    4
42243    4
Name: RegionNumber, dtype: int64

In [103]:
ens_clean["ScramType1"].unique()

array([0, 1], dtype=int64)

In [99]:
ens_clean["ScramType1"].replace(["N"],0, inplace = True)

In [101]:
ens_clean["ScramType2"].replace(["N"],0, inplace = True)
ens_clean["ScramType3"].replace(["N"],0, inplace = True)

In [102]:
ens_clean['ScramType1'].replace(['A/R', 'M/R', 'A', 'M'], 1, inplace = True)

In [104]:
ens_clean['ScramType2'].replace(['A/R', 'M/R', 'A', 'M'], 1, inplace = True)
ens_clean['ScramType3'].replace(['A/R', 'M/R', 'A', 'M'], 1, inplace = True)

In [105]:
ens_clean["ScramType2"].unique()

array([0, 1], dtype=int64)

In [106]:
ens_clean["ScramType3"].head(3)

EventNotificationNumber
42241    0
42242    0
42243    0
Name: ScramType3, dtype: int64

In [107]:
ens_clean["Scrams"] = ens_clean.ScramType1 + ens_clean.ScramType2 + ens_clean.ScramType3

In [108]:
ens_clean.head(3)

,SiteName,RegionNumber,EventDate,CfrDesc1,CfrDesc2,CfrDesc3,CfrDesc4,ScramType1,ScramType2,ScramType3,EventText,Year,Scrams
EventNotificationNumber,,,,,,,,,,,,,
42241,DUANE ARNOLD,3,2006-01-04,50.72(b)(3)(xiii) - LOSS COMM/ASMT/RESPONSE,NaN,NaN,NaN,0,0,0,PARTIAL LOSS OF EMERGENCY SIRENS,2006,0
42242,CALLAWAY,4,2006-01-04,50.72(b)(3)(ii)(B) - UNANALYZED CONDITION,NaN,NaN,NaN,0,0,0,UNANALYZED CONDITION RELATED TO EMERGENCY DIES...,2006,0
42243,RIVER BEND,4,2006-01-05,50.72(b)(3)(ii)(B) - UNANALYZED CONDITION,NaN,NaN,NaN,0,0,0,UNANALYZED CONDITION INVOLVING RCIC OPERATION ...,2006,0


In [109]:
ens_clean.shape

(6383, 13)

In [111]:
ens_final = ens_clean[["SiteName","RegionNumber","EventDate","Year","Scrams"]]

In [112]:
ens_final.head(3)

,SiteName,RegionNumber,EventDate,Year,Scrams
EventNotificationNumber,,,,,
42241,DUANE ARNOLD,3,2006-01-04,2006,0
42242,CALLAWAY,4,2006-01-04,2006,0
42243,RIVER BEND,4,2006-01-05,2006,0


In [113]:
ens_final.shape

(6383, 5)

In [123]:
ens_final["Scrams"].head(3)

EventNotificationNumber
42241    0
42242    0
42243    0
Name: Scrams, dtype: int64

In [114]:
ens_final["SiteName"].unique()

array(['DUANE ARNOLD', 'CALLAWAY', 'RIVER BEND', 'MILLSTONE', 'FERMI',
       'PALISADES', 'GRAND GULF', 'PRAIRIE ISLAND', 'CATAWBA',
       'SAN ONOFRE', 'BRUNSWICK', 'CLINTON', 'QUAD CITIES', 'HATCH',
       'INDIAN POINT', 'FARLEY', 'PERRY', 'HADDAM NECK', 'DAVIS BESSE',
       'SAINT LUCIE', 'COOPER', 'SUSQUEHANNA', 'OYSTER CREEK', 'DRESDEN',
       'MONTICELLO', 'VOGTLE', 'OCONEE', 'SOUTH TEXAS PROJECT',
       'WOLF CREEK', 'HOPE CREEK', 'BRAIDWOOD', 'SALEM', 'CRYSTAL RIVER',
       'FORT CALHOUN', 'BYRON', 'PALO VERDE', 'NINE MILE POINT',
       'BROWNS FERRY', 'SEABROOK', 'TURKEY POINT', 'LASALLE',
       'THREE MILE ISLAND', 'PILGRIM', 'POINT BEACH', 'SUMMER',
       'NORTH ANNA', 'SEQUOYAH', 'MCGUIRE', 'COLUMBIA', 'DIABLO CANYON',
       'BEAVER VALLEY', 'PEACH BOTTOM', 'CALVERT CLIFFS', 'FITZPATRICK',
       'KEWAUNEE', 'COOK', 'LIMERICK', 'GINNA', 'VERMONT YANKEE',
       'WATTS BAR', 'HARRIS', 'WATERFORD', 'SURRY',
       'ARKANSAS NUCLEAR ONE', 'LACROSSE', 'ROBINSON', 'CO

In [119]:
ens_final = ens_final[(ens_final.SiteName != "LACROSSE") & (ens_final.SiteName != "RANCHO SECO") & (ens_final.SiteName != 
                       "ZION") & (ens_final.SiteName != "BELLEFONTE") & (ens_final.SiteName != "HUMBOLDT BAY") & 
                      (ens_final.SiteName != "HADDAM NECK")]

In [120]:
ens_final.shape

(6362, 5)

In [121]:
ens_final["SiteName"].nunique()

65

In [140]:
first_scrams = ens_final.groupby(by=['SiteName', 'Year']).agg(ENCount = pd.NamedAgg(column = "RegionNumber", aggfunc ="count"),
                                                              ScramCount = pd.NamedAgg(column = 'Scrams', aggfunc = sum)).reset_index()

In [141]:
first_scrams.head()

,SiteName,Year,ENCount,ScramCount
0,ARKANSAS NUCLEAR ONE,2006,2,0
1,ARKANSAS NUCLEAR ONE,2007,2,1
2,ARKANSAS NUCLEAR ONE,2008,5,2
3,ARKANSAS NUCLEAR ONE,2009,12,5
4,ARKANSAS NUCLEAR ONE,2010,4,2


In [142]:
first_scrams.shape

(886, 4)

In [156]:
first_scrams["SiteName"].value_counts().tail(17)

SUSQUEHANNA       14
DAVIS BESSE       14
NORTH ANNA        13
SAN ONOFRE        13
SURRY             13
TURKEY POINT      13
POINT BEACH       13
CATAWBA           13
HARRIS            13
WATERFORD         13
FORT CALHOUN      13
SEABROOK          13
WATTS BAR         13
GINNA             13
VERMONT YANKEE    11
CRYSTAL RIVER     10
KEWAUNEE           9
Name: SiteName, dtype: int64

In [179]:
first_scrams[first_scrams["SiteName"] == "VERMONT YANKEE"]

,SiteName,Year,ENCount,ScramCount
821,VERMONT YANKEE,2006,5,0
822,VERMONT YANKEE,2007,7,1
823,VERMONT YANKEE,2008,5,0
824,VERMONT YANKEE,2009,2,0
825,VERMONT YANKEE,2010,8,1
826,VERMONT YANKEE,2011,8,0
827,VERMONT YANKEE,2012,1,0
828,VERMONT YANKEE,2013,5,0
829,VERMONT YANKEE,2014,2,0
830,VERMONT YANKEE,2016,1,0


In [180]:
first_scrams.drop([830,831,406,707,708,709,710,711,203,204,314,315,531], inplace = True)

In [181]:
first_scrams["SiteName"].value_counts().tail(17)

DAVIS BESSE       14
OYSTER CREEK      13
NORTH ANNA        13
SURRY             13
TURKEY POINT      13
CATAWBA           13
WATTS BAR         13
HARRIS            13
WATERFORD         13
SEABROOK          13
GINNA             13
POINT BEACH       13
FORT CALHOUN      11
VERMONT YANKEE     9
CRYSTAL RIVER      8
SAN ONOFRE         8
KEWAUNEE           8
Name: SiteName, dtype: int64

In [182]:
no_ens = pd.DataFrame([["NORTH ANNA", 2017, 0, 0], ["SURRY", 2007, 0, 0], ["TURKEY POINT", 2018, 0,0],
                       ["POINT BEACH", 2019, 0,0], ["CATAWBA", 2017, 0, 0], ["HARRIS", 2017, 0, 0], ["WATERFORD", 2007, 0, 0],
                       ["SEABROOK", 2013, 0, 0], ["WATTS BAR", 2007, 0, 0], ["GINNA", 2019, 0, 0]], 
                      columns=['SiteName', 'Year', "ENCount", "ScramCount"])

In [183]:
no_ens.head()

,SiteName,Year,ENCount,ScramCount
0,NORTH ANNA,2017,0,0
1,SURRY,2007,0,0
2,TURKEY POINT,2018,0,0
3,POINT BEACH,2019,0,0
4,CATAWBA,2017,0,0


In [192]:
first_scrams = first_scrams.append(no_ens, ignore_index = True)

In [193]:
first_scrams.shape

(883, 4)

In [194]:
df = first_scrams.sort_values(by =['SiteName', 'Year'])
df.shape

(883, 4)

In [196]:
df.tail()

,SiteName,Year,ENCount,ScramCount
868,WOLF CREEK,2015,8,2
869,WOLF CREEK,2016,5,0
870,WOLF CREEK,2017,2,0
871,WOLF CREEK,2018,4,0
872,WOLF CREEK,2019,3,0


In [197]:
df.to_excel('data.xlsx')